In [0]:
#This Code predict Label for SVF Issues 

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsSVF_20102019.csv')

y = list(data['Assignee']) 
x = list(data['Title_Description']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:80])
y_train = np.asarray(y_enc[:80])


x_test = np.asarray(x_enc[80:])
y_test = np.asarray(y_enc[80:])

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [40]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [41]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=30, batch_size=32)
    model.save_weights('./model.h5')

Epoch 1/30
80/80 [==============================] - 12s 153ms/step - loss: 2.3650 - acc: 0.6750
Epoch 2/30
80/80 [==============================] - 7s 85ms/step - loss: 2.0597 - acc: 0.8000
Epoch 3/30
80/80 [==============================] - 7s 85ms/step - loss: 1.7104 - acc: 0.8000
Epoch 4/30
80/80 [==============================] - 7s 85ms/step - loss: 1.3987 - acc: 0.8000
Epoch 5/30
80/80 [==============================] - 7s 86ms/step - loss: 1.1361 - acc: 0.8000
Epoch 6/30
80/80 [==============================] - 7s 85ms/step - loss: 0.9979 - acc: 0.8000
Epoch 7/30
80/80 [==============================] - 7s 85ms/step - loss: 0.9400 - acc: 0.8000
Epoch 8/30
80/80 [==============================] - 7s 86ms/step - loss: 0.9369 - acc: 0.8000
Epoch 9/30
80/80 [==============================] - 7s 85ms/step - loss: 0.9347 - acc: 0.8000
Epoch 10/30
80/80 [==============================] - 7s 85ms/step - loss: 0.9221 - acc: 0.8000
Epoch 11/30
80/80 [==============================] - 7s 8

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [47]:
y_test = np.asarray(y_enc[80:])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))



                     precision    recall  f1-score   support

 unsw-corg|yuleisui       0.00      0.00      0.00         2
           yuleisui       0.75      1.00      0.86         9
yuleisui|jcarlson23       0.00      0.00      0.00         1

           accuracy                           0.75        12
          macro avg       0.25      0.33      0.29        12
       weighted avg       0.56      0.75      0.64        12



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# New Section